#### import the relevant package

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

C:\ProgramData\anaconda3\envs\py310-TF\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load MNIST dataset
mnist_datasets, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_datasets['train'], mnist_datasets['test']

# Calculate the number of validation samples and cast to int64
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# Define a function to scale images
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

# Apply scaling to training and validation data
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

# Shuffle the scaled training and validation data
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

# Create validation data by taking a portion of the shuffled data
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# Create validation data by taking a portion of the shuffled data and apply reshaping
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
validation_data = validation_data.map(lambda x, y: (tf.reshape(x, shape=(-1, 784)), y))

BATCH_SIZE = 100

# # Batch the training data
# train_data = train_data.batch(BATCH_SIZE)

# # Batch the validation data
# validation_data = validation_data.batch(num_validation_samples)

# # Batch the test data
# test_data = test_data.batch(num_test_samples)

# # Get validation inputs and targets
# validation_inputs, validation_targets = next(iter(validation_data))

# Batch the training data and reshape it
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.map(lambda x, y: (tf.reshape(x, shape=(-1, 784)), y))

# Batch the validation data and reshape it
validation_data = validation_data.batch(num_validation_samples)
validation_inputs, validation_targets = next(iter(validation_data))
validation_inputs = tf.reshape(validation_inputs, shape=(-1, 784))

# Batch the test data and reshape it
test_data = test_data.batch(num_test_samples)
test_inputs, test_targets = next(iter(test_data))
test_inputs = tf.reshape(test_inputs, shape=(-1, 784))


### Model

#### Outline the model

In [3]:
input_size = 784
output_size = 10
hidden_layer_size = 70

# Create a Sequential model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(input_size,)),  # Input layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),  # Hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),  # Hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax')  # Output layer (returns probabilities)
])


### choose the optimizer and loss function

In [4]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

#### WHAT HAPPENS INSIDE AN EPOCH

1. At the beginning of each epoch, the training loss will be set to 0 
2. The algorithm will iterate over a preset number of batches, all from train_data
3. The weights and biases will be updated as many times as there are batches
4. We will get a value for the loss function, indicating how the training is going
5. We will also see a training accuracy 
6. At the end of the epoch, the algorithm will forward propagate the whole validation set



In [5]:
NUM_EPOCH = 15
model.fit(train_data, epochs=NUM_EPOCH, validation_data=(validation_inputs, validation_targets),verbose=2)

Epoch 1/15
540/540 - 7s - loss: 0.3601 - accuracy: 0.8991 - val_loss: 0.1939 - val_accuracy: 0.9435 - 7s/epoch - 12ms/step
Epoch 2/15
540/540 - 4s - loss: 0.1593 - accuracy: 0.9532 - val_loss: 0.1352 - val_accuracy: 0.9610 - 4s/epoch - 8ms/step
Epoch 3/15
540/540 - 6s - loss: 0.1159 - accuracy: 0.9655 - val_loss: 0.1201 - val_accuracy: 0.9665 - 6s/epoch - 11ms/step
Epoch 4/15
540/540 - 6s - loss: 0.0927 - accuracy: 0.9723 - val_loss: 0.1029 - val_accuracy: 0.9695 - 6s/epoch - 11ms/step
Epoch 5/15
540/540 - 6s - loss: 0.0768 - accuracy: 0.9769 - val_loss: 0.0856 - val_accuracy: 0.9733 - 6s/epoch - 11ms/step
Epoch 6/15
540/540 - 6s - loss: 0.0637 - accuracy: 0.9803 - val_loss: 0.0752 - val_accuracy: 0.9768 - 6s/epoch - 10ms/step
Epoch 7/15
540/540 - 5s - loss: 0.0546 - accuracy: 0.9843 - val_loss: 0.0627 - val_accuracy: 0.9797 - 5s/epoch - 10ms/step
Epoch 8/15
540/540 - 5s - loss: 0.0464 - accuracy: 0.9859 - val_loss: 0.0534 - val_accuracy: 0.9828 - 5s/epoch - 10ms/step
Epoch 9/15
540/54

### Test the model

In [7]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

313/313 [==============================] - 1s 2ms/step - loss: 0.0925 - accuracy: 0.9754


In [9]:
print('Test loss: {:.2f}, Test accuracy: {:.2f}%'.format(test_loss, test_accuracy * 100.0))


Test loss: 0.09, Test accuracy: 97.54%
